In [599]:
import numpy as np
import pandas as pd

In [600]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [601]:
df = pd.read_csv('covid_toy.csv')
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [602]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df.drop(columns=['has_covid']), df['has_covid'], test_size=0.2, random_state=42)

In [603]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [604]:
df[df['fever'].isnull() == True]

,age,gender,fever,cough,city,has_covid
5,84,Female,NaN,Mild,Bangalore,Yes
7,20,Female,NaN,Strong,Mumbai,Yes
10,75,Female,NaN,Mild,Delhi,No
19,42,Female,NaN,Strong,Bangalore,Yes
25,23,Male,NaN,Mild,Mumbai,No
29,34,Female,NaN,Strong,Mumbai,Yes
41,82,Male,NaN,Mild,Kolkata,Yes
56,71,Male,NaN,Strong,Kolkata,No
91,38,Male,NaN,Mild,Delhi,Yes
94,79,Male,NaN,Strong,Kolkata,Yes


## ***Aam Zindegi***


In [605]:
si = SimpleImputer()

# 1. Fit and transform the training data
x_train_fever = si.fit_transform(x_train[['fever']])

# 2. Only transform the test data (using the mean learned from x_train)
#    Note: usage of transform() instead of fit_transform()
#    Note: usage of x_test instead of x_train
x_test_fever = si.transform(x_test[['fever']]) 

x_train_fever.shape

(80, 1)

In [606]:
oe = OrdinalEncoder(categories=[['Mild', 'Strong']], dtype=int)
x_train_cough = oe.fit_transform(x_train[['cough']])
x_test_cough = oe.transform(x_test[['cough']])

x_train_cough.shape


(80, 1)

In [607]:
ohe = OneHotEncoder(drop='first', sparse_output=False)

x_train_gender_city = ohe.fit_transform(x_train[['gender', 'city']])
x_test_gender_city = ohe.transform(x_test[['gender', 'city']])

In [608]:
x_train.head()

,age,gender,fever,cough,city
55,81,Female,101.0,Mild,Mumbai
88,5,Female,100.0,Mild,Kolkata
26,19,Female,100.0,Mild,Kolkata
42,27,Male,100.0,Mild,Delhi
69,73,Female,103.0,Mild,Delhi


In [609]:
x_train_age = x_train.drop(columns=['gender', 'fever', 'city', 'cough']).values
x_test_age = x_test.drop(columns=['gender', 'fever', 'city', 'cough']).values

In [610]:
x_train_transform = np.concatenate((x_train_age, x_train_fever, x_train_gender_city, x_train_cough), axis=1)
x_test_transform = np.concatenate((x_test_age, x_test_fever, x_test_gender_city, x_test_cough), axis=1)

In [611]:
pd.DataFrame(x_train_transform).sample(5)

,0,1,2,3,4,5,6
21,74.0,102.0,1.0,0.0,0.0,1.0,0.0
58,79.0,101.0,1.0,0.0,1.0,0.0,1.0
40,71.0,101.0,1.0,0.0,1.0,0.0,1.0
22,20.0,101.0,0.0,0.0,0.0,1.0,1.0
63,34.0,101.0,0.0,0.0,0.0,1.0,1.0


In [612]:
from sklearn.compose import ColumnTransformer
transformer = ColumnTransformer(transformers=[
  ('tnf1', SimpleImputer(), ['fever']),
  ('tnf2', OrdinalEncoder(categories=[['Mild', 'Strong']]), ['cough']),
  ('tnf3', OneHotEncoder(sparse_output=False, drop='first'), ['gender', 'city'])
], remainder='passthrough')

In [613]:
x_train_transform = pd.DataFrame(transformer.fit_transform(x_train))
x_test_transform = pd.DataFrame(transformer.transform(x_test))

In [614]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

x_train_scaled = scaler.fit_transform(x_train_transform)
x_test_scaled = scaler.transform(x_test_transform)


In [615]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression(max_iter=1000)
model.fit(x_train_scaled, y_train)

y_pred = model.predict(x_test_scaled)

predictions = accuracy_score(y_test, y_pred)
predictions

0.45